In [93]:
import os, array
import numpy as np
import random
import math
from operator import itemgetter

In [7]:
imageNames = [name for name in os.listdir('./vanhateren_iml')]

In [8]:
imgs = []

for filename in imageNames:
    
    if filename == 'readIML.m':
        #skip this file
        continue
    
    fullPath = './vanhateren_iml/' + filename
    with open(fullPath, 'rb') as handle:
        s = handle.read()
    arr = array.array('H', s)
    arr.byteswap()
    img = np.array(arr, dtype='uint16').reshape(1024, 1536)
    imgs.append(img)
    
imgs = np.array(imgs)

In [15]:
sample = imgs[0]

In [16]:
sample.shape

(1024, 1536)

In [34]:
def extract_patches(img, xLength, yLength, patchLength, numPatches):
    
    patches = []
    
    random.seed(12345)
    
    for i in range(numPatches):
    
        xIndex = random.randint(0,xLength - patchLength)
        yIndex = random.randint(0,yLength - patchLength)

        patch = sample[xIndex:xIndex + patchLength, yIndex:yIndex + patchLength]
    
        patches.append(patch)
    
    return patches

In [35]:
patches  = extract_patches(sample, 1024, 1536, 3, 5000)

In [88]:
def compute_D_norm(patch, D):
    
    #Turn patch from 3x3 matrix to flat vector
    patch = patch.flatten()

    xTD = np.dot(patch.T,D)
    
    xTDx = np.dot(xTD,patch)

    return math.sqrt(xTDx)
        

In [89]:
def pre_process_patches(patches):
    #Returns a tuple of the form
    #(processedPatch, DNorm)
    
    processedPatchesAndDNorm = []
    
    D = np.matrix([
        [2,-1,0,-1,0,0,0,0,0],
        [-1,3,-1,0,-1,0,0,0,0],
        [0,-1,2,0,0,-1,0,0,0],
        [-1,0,0,3,-1,0,-1,0,0],
        [0,-1,0,-1,4,-1,0,-1,0],
        [0,0,-1,0,-1,3,0,0,-1],
        [0,0,0,-1,0,0,2,-1,0],
        [0,0,0,0,-1,0,-1,3,-1],
        [0,0,0,0,0,-1,0,-1,2]
    ])
    
    for patch in patches:
        
        logVals = []
        
        for row in patch:
            
            for pix in row:
            
                logPix = math.log(pix)
                logVals.append(logPix)
        
        patchLength = patch.shape[0]
        logPatch = np.array(logVals).reshape(patchLength,patchLength)
        
        avg = np.mean(logPatch)
        
        averagedPatch = np.array([val - avg for val in logPatch]).reshape(patchLength,patchLength)
        
        DNorm = compute_D_norm(averagedPatch, D)
        
        tup = (averagedPatch,DNorm)
        processedPatchesAndDNorm.append(tup)
            
    return processedPatchesAndDNorm
         

In [90]:
patchesAndNorm = pre_process_patches(patches)

In [102]:
sortedLst = sorted(patchesAndNorm,key=itemgetter(1),reverse=True)

In [109]:
top20Index = int(5000 * 0.2)

selectedPatchesAndNorms = sortedLst[:top20Index]

In [111]:
def normalizeVectors(patchesAndNorms):
    #Takes a list of tuples of the form
    #(patch/vector,DNorm)
    
    normalizedPatches = []
    for patchAndNorm in patchesAndNorms:
        
        DNorm = patchAndNorm[1]
        
        normalizedPatch = []
        for pix in patchAndNorm[0]:
            normalized = pix / DNorm
            
            normalizedPatch.append(normalized)
        
        normalizedPatch = np.array(normalizedPatch)
            
        normalizedPatches.append(normalizedPatch)
            
    return np.array(normalizedPatches)   

In [112]:
normalizedPatches = normalizeVectors(selectedPatchesAndNorms)